# The Cancer Gene Atlas Cancer (TCGA) - Thymoma (THYM)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.5 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.3.1
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.2
pandas 0.22.0
clustergrammer_widget 1.9.0
seaborn 0.8.1
Fri May 11 15:13:11 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Thymoma [THYM]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-3G-AB0T,394,THYM,TCGA,ACAAED1A-2667-4C56-919B-DAF5574371BB,3G,45.0,NaN,431,Nationwide Children's Hospital,NaN,...,33.0,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-3G-AB14,395,THYM,TCGA,6E98A600-252D-442F-9372-DDC7C5338AE7,3G,51.0,NaN,431,Nationwide Children's Hospital,NaN,...,17.5,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-3G-AB19,396,THYM,TCGA,81FDC97F-8BAD-4286-9D36-9A887C6B59A1,3G,76.0,NaN,431,Nationwide Children's Hospital,NaN,...,50.4,NaN,NaN,"Yes, History of Synchronous/Bilateral Malignancy",Kidney,NaN,Synchronous Malignancy,NaN,NaN,NaN
TCGA-3G-AB0Q,397,THYM,TCGA,67FF3AA1-9D52-4DC8-8501-ECD0D37A8735,3G,66.0,NaN,431,Nationwide Children's Hospital,NaN,...,21.9,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-3G-AB0O,398,THYM,TCGA,3F19D238-1B55-4530-A9E2-8FDA644D45A6,3G,47.0,NaN,431,Nationwide Children's Hospital,NaN,...,33.5,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(124, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_THYM_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,39bf7fed-5083-4e32-a6dd-55fec205fbb7,b6e769d9-d151-44dd-8465-697bbf943947,fae957a6-9976-4995-948f-31041b9f700d,323a5e02-0900-4ace-bd35-da74448c278c,b10d3ac7-69d1-4a1c-99fe-34cff1fa2616,a5fde6b7-6dae-42eb-9500-2b5f5013b2f0,40dc6872-6ee6-4dd5-ad36-0ef5a9699237,0332f256-c042-4d58-b99f-8f632775f051,a5321450-e30c-40cb-a405-903368128bc6,5073b43b-2743-4eec-8826-567b8a5bf5ed,...,e0bd4e1e-4e25-466d-b4a3-c05a235ee342,cd1db64b-f217-4d14-affd-811884c9d70f,840b2058-8abf-4f95-afc7-aa2edc2f3561,fbe56fd9-a1b6-48fb-bae2-3a7e12c3ca71,9e1c0b89-626e-4bc8-b244-38b52962ab1c,9509a5c7-d299-46ff-9aa1-2bc1e48036f0,c489221f-a91d-49c4-912a-eb170ab7e1cd,9eef49d7-2006-434f-b4f0-d6be39c392a3,4ab5e9f7-63f4-4b6a-b280-57ba6ff41499,9ff75aef-c890-4758-9c1e-7e2aa130e500
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,198.1730,123.542000,183.503000,85.61130,116.75000,115.79200,179.36100,91.5287,188.031000,222.92700,...,248.77600,167.0730,192.67200,126.08200,194.3460,193.696000,214.213000,220.212000,250.482000,123.421000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,25.6475,39.449900,12.007600,38.99900,23.35940,28.80160,18.69520,50.2186,29.634400,15.08590,...,15.25190,24.8601,14.76890,32.02770,24.4234,9.180050,4.336340,0.935714,0.881002,0.000000
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,17.3265,13.915200,13.016200,12.02070,12.96330,9.43857,12.69560,17.2435,39.574500,12.62700,...,19.39390,11.0384,12.34940,7.10431,11.6523,10.759100,10.438600,11.800700,6.479270,3.748280
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,28.2523,27.121200,24.310300,23.65810,24.10370,28.14190,25.97560,32.3988,58.350200,38.05380,...,8.78116,40.4555,30.16810,46.92810,26.5179,33.082500,38.695800,39.673200,23.323200,16.508700
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,14.3207,0.482857,0.450488,6.29652,8.29746,2.67351,2.71058,5.6795,0.386402,4.00351,...,3.55947,3.7058,3.40402,3.19404,11.0175,0.524533,0.079467,0.000000,0.155325,0.236389


In [19]:
matrix.shape

(199169, 122)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

In [22]:
matrix.shape

# Map Sample Meta Data to Sample ID

In [23]:
getSampleID(matrix, meta)

In [24]:
matrix.head()

# Map Gene Symbol to Gene ID

In [25]:
getGenes(matrix)

In [26]:
matrix.head()

,TCGA-ZC-AAAA-01,TCGA-ZB-A962-01,TCGA-ZB-A96B-01,TCGA-ZB-A96C-01,TCGA-ZB-A96H-01,TCGA-ZB-A96K-01,TCGA-XH-A853-01,TCGA-ZB-A96L-01,TCGA-ZB-A96V-01,TCGA-ZB-A96R-01,...,TCGA-3G-AB14-01,TCGA-4V-A9QT-01,TCGA-4V-A9QJ-01,TCGA-4V-A9QM-01,TCGA-X7-A8D7-01,TCGA-4V-A9QX-01,TCGA-ZB-A96A-01,TCGA-ZC-AAAH-01,TCGA-X7-A8M7-01,TCGA-XU-AAXZ-01
ARF5,198.1730,123.542000,183.503000,85.61130,116.75000,115.79200,179.36100,91.5287,188.031000,222.92700,...,248.77600,167.0730,192.67200,126.08200,194.3460,193.696000,214.213000,220.212000,250.482000,123.421000
M6PR,25.6475,39.449900,12.007600,38.99900,23.35940,28.80160,18.69520,50.2186,29.634400,15.08590,...,15.25190,24.8601,14.76890,32.02770,24.4234,9.180050,4.336340,0.935714,0.881002,0.000000
ESRRA,17.3265,13.915200,13.016200,12.02070,12.96330,9.43857,12.69560,17.2435,39.574500,12.62700,...,19.39390,11.0384,12.34940,7.10431,11.6523,10.759100,10.438600,11.800700,6.479270,3.748280
FKBP4,28.2523,27.121200,24.310300,23.65810,24.10370,28.14190,25.97560,32.3988,58.350200,38.05380,...,8.78116,40.4555,30.16810,46.92810,26.5179,33.082500,38.695800,39.673200,23.323200,16.508700
CYP26B1,14.3207,0.482857,0.450488,6.29652,8.29746,2.67351,2.71058,5.6795,0.386402,4.00351,...,3.55947,3.7058,3.40402,3.19404,11.0175,0.524533,0.079467,0.000000,0.155325,0.236389


# Histogram of First Sample

In [27]:
sns.distplot(matrix.iloc[:, 0]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [28]:
sns.distplot(matrix.iloc[0, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Unfiltered Matrix to File

In [29]:
filename = path+'tcga_thym_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [30]:
normalized_matrix = matrix.copy()

In [31]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining missing values to matrix mean

In [32]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [33]:
normalized_matrix.head()

,TCGA-ZC-AAAA-01,TCGA-ZB-A962-01,TCGA-ZB-A96B-01,TCGA-ZB-A96C-01,TCGA-ZB-A96H-01,TCGA-ZB-A96K-01,TCGA-XH-A853-01,TCGA-ZB-A96L-01,TCGA-ZB-A96V-01,TCGA-ZB-A96R-01,...,TCGA-3G-AB14-01,TCGA-4V-A9QT-01,TCGA-4V-A9QJ-01,TCGA-4V-A9QM-01,TCGA-X7-A8D7-01,TCGA-4V-A9QX-01,TCGA-ZB-A96A-01,TCGA-ZC-AAAH-01,TCGA-X7-A8M7-01,TCGA-XU-AAXZ-01
A1BG,0.042286,0.030405,0.042286,0.029831,0.042286,0.042286,0.042286,0.037507,0.042286,0.042286,...,0.029342,0.042286,0.035145,0.042286,0.042286,0.037290,0.042286,0.042286,0.019455,0.042286
A1CF,0.025813,0.012392,0.002811,0.019622,0.008834,0.009624,0.002787,0.022726,0.007976,0.026242,...,0.005543,0.004392,0.007053,0.012224,0.005296,0.010594,0.010578,0.007071,0.012530,0.011057
A2M,39.460600,83.626600,23.951700,105.027000,68.289200,92.322800,40.649600,88.978300,70.238100,25.024900,...,19.439700,8.171840,46.360300,75.875500,18.098700,19.723300,26.375900,18.040800,11.152100,23.440100
A2ML1,0.104590,0.095342,0.006385,0.164766,0.133021,0.087837,0.067715,0.130804,0.632512,0.225469,...,0.161914,0.016450,0.080183,0.075842,0.310775,0.059840,0.061518,0.085766,0.019069,0.102844
A3GALT2,0.077250,0.307635,0.063346,0.026001,0.094551,0.204163,0.065394,0.041064,0.065394,0.065394,...,0.065394,0.016832,0.065394,0.032127,0.065394,0.032901,0.065394,0.033771,0.065394,0.065394


In [34]:
normalized_matrix.shape

(19030, 122)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [35]:
samples = normalized_matrix.columns.tolist()

In [36]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [37]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [38]:
normalized_matrix.columns = samples

In [39]:
normalized_matrix.head()

,TCGA-ZC-AAAA-01,TCGA-ZB-A962-01,TCGA-ZB-A96B-01,TCGA-ZB-A96C-01,TCGA-ZB-A96H-01,TCGA-ZB-A96K-01,TCGA-XH-A853-01,TCGA-ZB-A96L-01,TCGA-ZB-A96V-01,TCGA-ZB-A96R-01,...,TCGA-3G-AB14-01,TCGA-4V-A9QT-01,TCGA-4V-A9QJ-01,TCGA-4V-A9QM-01,TCGA-X7-A8D7-01,TCGA-4V-A9QX-01,TCGA-ZB-A96A-01,TCGA-ZC-AAAH-01,TCGA-X7-A8M7-01,TCGA-XU-AAXZ-01
A1BG,-1.168019,-1.296581,-1.161503,-1.303939,-1.162148,-1.154194,-1.168381,-1.198320,-1.170039,-1.168322,...,-1.324929,-1.169019,-1.229565,-1.168323,-1.156244,-1.204309,-1.149717,-1.121938,-1.473776,-1.089642
A1CF,-1.382375,-1.686373,-2.338910,-1.485864,-1.842169,-1.797023,-2.349456,-1.415906,-1.894438,-1.375518,...,-2.048641,-2.152507,-1.927069,-1.707300,-2.058509,-1.750837,-1.751516,-1.898640,-1.664846,-1.672187
A2M,1.801946,2.142812,1.591635,2.242690,2.046006,2.184917,1.814478,2.176851,2.050335,1.603853,...,1.496270,1.117103,1.890722,2.085581,1.475206,1.519074,1.645292,1.508120,1.284556,1.654120
A2ML1,-0.774727,-0.800249,-1.982508,-0.561744,-0.664425,-0.836713,-0.963891,-0.655812,0.004831,-0.441433,...,-0.583135,-1.579046,-0.871343,-0.914610,-0.289996,-0.998911,-0.986915,-0.814820,-1.482473,-0.703661
A3GALT2,-0.906318,-0.291496,-0.985983,-1.363620,-0.812678,-0.470415,-0.979041,-1.158974,-0.980699,-0.978982,...,-0.976881,-1.569089,-0.959886,-1.287649,-0.966904,-1.258695,-0.960378,-1.219590,-0.947263,-0.900302


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [40]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [41]:
normalized_matrix.head()

,TCGA-ZC-AAAA-01,TCGA-ZB-A962-01,TCGA-ZB-A96B-01,TCGA-ZB-A96C-01,TCGA-ZB-A96H-01,TCGA-ZB-A96K-01,TCGA-XH-A853-01,TCGA-ZB-A96L-01,TCGA-ZB-A96V-01,TCGA-ZB-A96R-01,...,TCGA-3G-AB14-01,TCGA-4V-A9QT-01,TCGA-4V-A9QJ-01,TCGA-4V-A9QM-01,TCGA-X7-A8D7-01,TCGA-4V-A9QX-01,TCGA-ZB-A96A-01,TCGA-ZC-AAAH-01,TCGA-X7-A8M7-01,TCGA-XU-AAXZ-01
A1BG,-1.171496,-1.325517,-1.127144,-1.404543,-1.222999,-1.224244,-1.138664,-1.285183,-1.118503,-1.100652,...,-1.342792,-1.028648,-1.221443,-1.139313,-1.099541,-1.147386,-1.068207,-1.027792,-1.428650,-1.060309
A1CF,-1.391866,-1.712655,-2.416156,-1.584307,-1.901343,-1.878327,-2.466491,-1.514897,-1.852450,-1.330385,...,-2.067569,-1.990354,-1.912641,-1.671056,-2.051729,-1.672625,-1.646002,-1.823494,-1.617487,-1.650326
A2M,1.827962,2.062216,1.594614,2.188625,2.035805,2.141302,1.781292,2.061344,2.075428,1.602718,...,1.470916,1.149771,1.849393,2.048126,1.442974,1.506063,1.672152,1.636068,1.464030,1.864973
A2ML1,-0.768769,-0.869145,-1.985462,-0.712406,-0.772573,-0.931406,-0.938380,-0.806447,0.068704,-0.310788,...,-0.620950,-1.459222,-0.866996,-0.902171,-0.183701,-0.935273,-0.904307,-0.670381,-1.442437,-0.567084
A3GALT2,-0.905669,-0.418533,-0.951562,-1.467999,-0.907001,-0.601308,-0.950195,-1.243490,-0.913776,-0.898615,...,-0.999658,-1.449809,-0.951335,-1.263067,-0.890415,-1.205429,-0.878421,-1.153496,-0.827640,-0.833513


# Normalize Matrix (z-score the rows)

In [42]:
uf.zscore(normalized_matrix, 'row')

In [43]:
normalized_matrix.head()

,TCGA-ZC-AAAA-01,TCGA-ZB-A962-01,TCGA-ZB-A96B-01,TCGA-ZB-A96C-01,TCGA-ZB-A96H-01,TCGA-ZB-A96K-01,TCGA-XH-A853-01,TCGA-ZB-A96L-01,TCGA-ZB-A96V-01,TCGA-ZB-A96R-01,...,TCGA-3G-AB14-01,TCGA-4V-A9QT-01,TCGA-4V-A9QJ-01,TCGA-4V-A9QM-01,TCGA-X7-A8D7-01,TCGA-4V-A9QX-01,TCGA-ZB-A96A-01,TCGA-ZC-AAAH-01,TCGA-X7-A8M7-01,TCGA-XU-AAXZ-01
A1BG,-0.137013,-1.544613,0.268320,-2.266832,-0.607703,-0.619082,0.163035,-1.176004,0.347285,0.510431,...,-1.702487,1.168470,-0.593482,0.157102,0.520583,0.083324,0.806941,1.176298,-2.487144,0.879125
A1CF,1.360063,0.000000,-2.982662,0.544163,-0.799988,-0.702406,-3.196066,0.838445,-0.592692,1.620728,...,-1.504742,-1.177371,-0.847889,0.176372,-1.437583,0.169719,0.282593,-0.469926,0.403489,0.264259
A2M,0.244783,1.024755,-0.532175,1.445649,0.936817,1.288079,0.089389,1.021851,1.068747,-0.505190,...,-0.944041,-2.013326,0.316139,0.977842,-1.037075,-0.827015,-0.274005,-0.394148,-0.966969,0.368013
A2ML1,0.388186,-0.037964,-4.777317,0.627473,0.372034,-0.302296,-0.331905,0.228219,3.943698,2.332556,...,1.015752,-2.543152,-0.028840,-0.178178,2.872105,-0.318713,-0.187246,0.805892,-2.471892,1.244444
A3GALT2,0.348490,2.554176,0.140691,-2.197666,0.342459,1.726596,0.146879,-1.181120,0.311780,0.380426,...,-0.077081,-2.115304,0.141720,-1.269763,0.417555,-1.008784,0.471864,-0.773639,0.701795,0.675202


# Merge Like Column (by taking the mean)

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [45]:
normalized_matrix.shape

(19030, 122)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [46]:
uf.mapgenesymbols(normalized_matrix)

In [47]:
normalized_matrix.shape

# Merge Duplicate Genes By Rows

In [48]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [49]:
normalized_matrix.shape

# Histogram of First Sample

In [50]:
sns.distplot(normalized_matrix.iloc[:, 10]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [51]:
sns.distplot(normalized_matrix.iloc[10, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Filtered Matrix

In [52]:
filename = path+'tcga_thym_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [53]:
gene_list = uf.createGeneList(normalized_matrix)

In [54]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [55]:
gene_list.shape

(18514, 2)

# Save Gene List

In [56]:
filename = path+'tcga_thym_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [57]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [58]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [59]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-3G-AB0O-01,47,MALE,NaN,NaN,NaN,NaN,Thymoma; Type B1,Primary Solid Tumor,Thymus,NaN
TCGA-3G-AB0Q-01,66,MALE,NaN,NaN,NaN,NaN,Thymoma; Type A,Primary Solid Tumor,Thymus,NaN
TCGA-3G-AB0T-01,45,MALE,NaN,NaN,NaN,NaN,Thymoma; Type B1,Primary Solid Tumor,Anterior Mediastinum,NaN
TCGA-3G-AB14-01,51,MALE,NaN,NaN,NaN,NaN,Thymoma; Type B2,Primary Solid Tumor,Thymus,NaN
TCGA-3G-AB19-01,76,FEMALE,NaN,NaN,NaN,NaN,Thymoma; Type A,Primary Solid Tumor,Thymus,NaN


In [60]:
attribute_list.shape

(122, 10)

# Save Attribute List

In [61]:
filename = path+'tcga_thym_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [62]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [63]:
standard_matrix.head()

,TCGA-3G-AB0O-01,TCGA-3G-AB0Q-01,TCGA-3G-AB0T-01,TCGA-3G-AB14-01,TCGA-3G-AB19-01,TCGA-3Q-A9WF-01,TCGA-3S-A8YW-01,TCGA-3S-AAYX-01,TCGA-3T-AA9L-01,TCGA-4V-A9QI-01,...,TCGA-ZB-A96P-01,TCGA-ZB-A96Q-01,TCGA-ZB-A96R-01,TCGA-ZB-A96V-01,TCGA-ZC-AAA7-01,TCGA-ZC-AAAA-01,TCGA-ZC-AAAF-01,TCGA-ZC-AAAH-01,TCGA-ZL-A9V6-01,TCGA-ZT-A8OM-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.015259,0.985331,0.722608,-0.834552,-0.277518,-0.048057,0.427324,-0.900062,0.066701,0.640543,...,-0.867311,-0.670687,0.394572,0.296193,-0.064441,-0.146397,0.378194,0.771886,-0.556021,0.689740
A1CF,-0.311818,-0.918093,0.491329,-0.705074,0.032318,-0.131638,-0.852569,0.950885,0.770236,0.638878,...,0.556881,0.540500,0.918031,-0.344666,-0.541277,0.868784,-0.524871,-0.262753,0.885151,0.425693
A2M,-0.709379,0.880895,-0.692979,-0.774900,0.716665,-0.152326,-0.643887,-0.955102,-0.676593,-0.987795,...,-0.922357,0.487040,-0.463669,0.683818,0.306683,0.159091,-0.627497,-0.332531,0.897304,0.257484
A2ML1,0.339427,-0.217899,-0.496382,0.569013,-0.578332,0.323027,0.963013,0.224633,-0.250607,-0.742184,...,0.077151,-0.561933,0.815214,0.913753,0.995822,0.257484,-0.955102,0.503433,-0.430817,-0.611119
A3GALT2,-0.751513,-0.915311,0.690883,-0.046935,0.035082,0.018665,0.494109,-0.882532,0.149749,0.198893,...,0.280909,-0.325457,0.412064,0.264571,-0.112473,0.362921,-0.620456,-0.440186,-0.194363,0.674471


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [64]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [65]:
filename = path+'tcga_thym_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [66]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [67]:
tertiary_matrix.head()

,TCGA-3G-AB0O-01,TCGA-3G-AB0Q-01,TCGA-3G-AB0T-01,TCGA-3G-AB14-01,TCGA-3G-AB19-01,TCGA-3Q-A9WF-01,TCGA-3S-A8YW-01,TCGA-3S-AAYX-01,TCGA-3T-AA9L-01,TCGA-4V-A9QI-01,...,TCGA-ZB-A96P-01,TCGA-ZB-A96Q-01,TCGA-ZB-A96R-01,TCGA-ZB-A96V-01,TCGA-ZC-AAA7-01,TCGA-ZC-AAAA-01,TCGA-ZC-AAAF-01,TCGA-ZC-AAAH-01,TCGA-ZL-A9V6-01,TCGA-ZT-A8OM-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,0,0,0,0,0,-1,0,-1,...,0,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,-1,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [68]:
filename = path+'tcga_thym_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [69]:
name = 'tcga_thym_gene_up_set'

In [70]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [71]:
name = 'tcga_thym_gene_down_set'

In [72]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [73]:
name = 'tcga_thym_attribute_up_set'

In [74]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [75]:
name = 'tcga_thym_attribute_down_set'

In [76]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity Matrix

In [77]:
attribute_normalized_matrix = normalized_matrix.copy()

In [78]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [79]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [80]:
attribute_similarity_matix.head()

,"(Donor:TCGA-3G-AB0O, Age:47.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B1, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-3G-AB0Q, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type A, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-3G-AB0T, Age:45.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B1, Tissue:Anterior Mediastinum, Tumor:nan, Gradenan)","(Donor:TCGA-3G-AB14, Age:51.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B2, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-3G-AB19, Age:76.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type A, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-3Q-A9WF, Age:71.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B2, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-3S-A8YW, Age:63.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type C, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-3S-AAYX, Age:55.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B1, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-3T-AA9L, Age:31.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B1, Tissue:Anterior Mediastinum, Tumor:nan, Gradenan)","(Donor:TCGA-4V-A9QI, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B2, Tissue:Thymus, Tumor:nan, Gradenan)",...,"(Donor:TCGA-ZB-A96P, Age:73.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B1, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-ZB-A96Q, Age:49.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type AB, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-ZB-A96R, Age:38.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B2, Tissue:Anterior Mediastinum, Tumor:nan, Gradenan)","(Donor:TCGA-ZB-A96V, Age:44.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type C, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-ZC-AAA7, Age:63.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type C, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-ZC-AAAA, Age:39.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B2, Tissue:Anterior Mediastinum, Tumor:nan, Gradenan)","(Donor:TCGA-ZC-AAAF, Age:74.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type AB, Tissue:Anterior Mediastinum, Tumor:nan, Gradenan)","(Donor:TCGA-ZC-AAAH, Age:52.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B2, Tissue:Anterior Mediastinum, Tumor:nan, Gradenan)","(Donor:TCGA-ZL-A9V6, Age:61.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type AB, Tissue:Thymus, Tumor:nan, Gradenan)","(Donor:TCGA-ZT-A8OM, Age:73.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type A, Tissue:Anterior Mediastinum, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-3G-AB0O, Age:47.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B1, Tissue:Thymus, Tumor:nan, Gradenan)",1.000000,-0.191148,0.025751,0.050985,-0.154474,0.203430,0.024463,0.346521,0.285146,0.254694,...,0.232348,0.011299,0.335102,0.017375,-0.064634,0.129775,0.041613,0.049870,-0.114335,-0.017932
"(Donor:TCGA-3G-AB0Q, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type A, Tissue:Thymus, Tumor:nan, Gradenan)",-0.191148,1.000000,-0.089889,0.114835,0.266599,-0.131588,0.035558,-0.233281,-0.143512,-0.180768,...,-0.275833,0.066672,-0.131499,-0.013413,0.032621,-0.046565,0.040683,-0.117768,0.232175,-0.040495
"(Donor:TCGA-3G-AB0T, Age:45.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Thymoma; Type B1, Tissu

# Save Attribute Similarity Matrix

In [81]:
filename = path+'tcga_thym_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [82]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

clustergrammer_widget(network='{"row_nodes": [{"cat_4_index": 52, "cat_5_index": 27, "rank": 58, "cat-6": "Tum…

# Create Gene Similarity Matrix

In [83]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [84]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.184109,-0.117457,-0.084025,0.001008,0.046981,-0.041649,-0.000203,-0.188590,0.118889,...,-0.118753,0.095243,-0.334821,-0.309588,-0.101877,0.049035,-0.281251,0.033716,-0.281720,-0.289276
A1CF,-0.184109,1.000000,0.053686,0.006252,0.063628,-0.184644,0.047041,-0.062193,-0.080194,-0.187282,...,0.150781,-0.009216,0.161239,0.109609,-0.056679,0.072892,0.171054,-0.019799,0.255835,0.275135
A2M,-0.117457,0.053686,1.000000,0.119277,0.118510,0.551869,-0.075183,-0.338847,0.013067,-0.206280,...,-0.513977,-0.696920,0.359487,0.252160,-0.052376,-0.267437,0.292846,0.367322,-0.012784,0.183143
A2ML1,-0.084025,0.006252,0.119277,1.000000,-0.088577,-0.119083,-0.078190,-0.167583,0.385394,-0.203613,...,-0.059806,-0.102901,0.156571,0.114032,0.080662,0.414647,-0.058791,-0.094905,0.180145,-0.061351
A3GALT2,0.001008,0.063628,0.118510,-0.088577,1.000000,0.090058,0.025354,-0.022883,-0.122722,0.153565,...,0.017860,-0.037041,-0.149436,-0.165900,-0.050271,-0.025340,-0.013085,-0.013936,-0.127710,0.086265


# Save Gene Similarity Matrix

In [85]:
filename = path+'tcga_thym_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [86]:
name = 'tcga_thym_gene_attribute_edge_list'

In [87]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  122 Out of 122   

 The number of statisticaly relevent gene-attribute associations is: 112695
